In [1]:
DATE = "2021-09-11"
TASK = "annealing-correction-L100-K7"
DIR = mkpath("$(homedir())/$(DATE)-$(TASK)")

"/home/jovyan/2021-09-11-annealing-correction-L100-K7"

In [2]:
import Pkg
pkgs = [
    "BioAlignments",
    "BioSequences",
    "Clustering",
#     "CodecZlib",
#     "Colors",
#     "Combinatorics",
    "DataFrames",
    "DataStructures",
    "Dates",
#     "DelimitedFiles",
    "Distances",
#     "Distributions",
#     "EzXML",
    "FASTX",
#     "GFF3",
    "GraphPlot",
#     "HTTP",
#     "Impute",
#     "JSON",
    "Graphs",
#     "LSHFunctions",
#     "Measures",
    "MetaGraphs",
    "https://github.com/cjprybol/Mycelia.git",
#     "NumericIO",
#     "PlotlyJS",
#     "Plots",
    "Primes",
#     "Printf",
    "ProgressMeter",
    "Random",
    "Revise",
    "SparseArrays",
    "Statistics",
    "StatsBase",
    "StatsPlots",
#     "StringDistances",
    "uCSV",
#     "XLSX",
]

unregistered_packages = filter(pkg -> occursin(r"(^https|git$)", pkg), pkgs)
registered_packages = setdiff(pkgs, unregistered_packages)

for pkg in registered_packages
    try
        eval(Meta.parse("import $(pkg)"))
    catch
        Pkg.add(pkg)
        Pkg.build(pkg)
        eval(Meta.parse("import $(pkg)"))
    end
end

for pkg_url in unregistered_packages
    pkg_name = replace(basename(pkg_url), ".git" => "")
    try
        eval(Meta.parse("import $(pkg_name)"))
    catch
        Pkg.develop(url=pkg_url)
        Pkg.build(pkg_name)
        eval(Meta.parse("import $(pkg_name)"))
    end
end

In [3]:
function kmer_index_and_orientation_to_kmer(graph, kmer_index, orientation)
    kmer = graph.vprops[kmer_index][:kmer]
    if !orientation
        kmer = BioSequences.reverse_complement(kmer)
    end
    return kmer
end

kmer_index_and_orientation_to_kmer (generic function with 1 method)

In [4]:
function determine_edge_probabilities(graph)
    nv = Graphs.nv(graph)
    edge_probabilities = SparseArrays.spzeros(nv, nv)
    for v in 1:nv
        neighbors = Graphs.neighbors(graph, v)
        @assert issorted(neighbors)
        likelihoods = zeros(length(neighbors))
        for (i, neighbor) in enumerate(neighbors)
            if v <= neighbor
                edge = Graphs.Edge(v, neighbor)
            else
                edge = Graphs.Edge(neighbor, v)
            end
            @assert Graphs.has_edge(graph, edge)
            likelihoods[i] = graph.eprops[edge][:count]     
        end
        likelihoods = likelihoods ./ sum(likelihoods)
#         @show likelihoods
#         @show neighbors
        for (neighbor, likelihood) in zip(neighbors, likelihoods)
            edge_probabilities[v, neighbor] = likelihood
        end
    end
    
    for source in 1:size(edge_probabilities, 1)
        destinations = findall(edge_probabilities[source, :] .> 0)
        destination_counts = [graph.vprops[dest][:count] for dest in destinations]
        destination_likelihoods = destination_counts ./ sum(destination_counts)
        for (dest, likelihood) in zip(destinations, destination_likelihoods)
            edge_probabilities[source, dest] *= likelihood
        end
        edge_probabilities[source, :] ./= sum(edge_probabilities[source, :])
        @assert abs(1-sum(edge_probabilities[source, :])) <= eps(Float64)
    end
    
    return edge_probabilities
end

determine_edge_probabilities (generic function with 1 method)

In [5]:
function orient_path(graph, kmers, path, opening_orientation, opening_kmer, closing_orientation)
    vertices = path
    path_orientations = [opening_orientation]
    path_kmers = [opening_kmer]
    @show opening_orientation
    for (i, vertex) in enumerate(vertices[2:end])
        @show i, vertex, last(path_orientations)
        viable_neighbors = typeof(opening_kmer)[]
        viable_neighbor = nothing
        viable_orientations = Bool[]
        viable_orientation = nothing
        for neighbor in BioSequences.neighbors(last(path_kmers))
            @show "considering going from $(last(path_kmers)) to $(neighbor)"
#             @show "considering going from $(last(path_kmers)) to $(neighbor)"
            canonical_neighbor = BioSequences.canonical(neighbor)
            canonical_neighbor_index_range = searchsorted(kmers, canonical_neighbor)
#             @show canonical_neighbor_index_range
            if length(canonical_neighbor_index_range) == 1
                @show canonical_neighbor_index = first(canonical_neighbor_index_range)
                if canonical_neighbor_index == vertex
                    @show "hit, selecting $neighbor"
                    viable_neighbor = neighbor
                    viable_orientation = neighbor == canonical_neighbor
                end
            end
        end
        if (viable_neighbor != nothing) && (viable_orientation != nothing)
            push!(path_kmers, viable_neighbor)
            push!(path_orientations, viable_orientation)
        else
            return nothing
            # not a viable path, need to look towards the next miss
        end
    end
    return vertices, path_orientations, path_kmers
end

orient_path (generic function with 1 method)

In [6]:
function take_a_walk(graph, edge_likelihoods, kmers, kmer, walk_length)
    walk = [kmer]
    current_kmer_indices = searchsorted(kmers, BioSequences.canonical(last(walk)))
    @assert length(current_kmer_indices) == 1
    current_kmer_index = first(current_kmer_indices)
    while length(walk) < walk_length
        viable_neighbors = Tuple{Int, Bool}[]
        for neighbor in BioSequences.neighbors(kmer)
            canonical_neighbor = BioSequences.canonical(neighbor)
            neighbor_is_canonical = neighbor == canonical_neighbor
            neighbor_vertex_range = searchsorted(kmers, canonical_neighbor)
            if !isempty(neighbor_vertex_range)
                neighbor_vertex = first(neighbor_vertex_range)
                if Graphs.has_edge(graph, current_kmer_index, neighbor_vertex)
                    push!(viable_neighbors, (neighbor_vertex, neighbor_is_canonical))
                end
            end
        end
        if isempty(viable_neighbors)
            return walk
        elseif length(viable_neighbors) == 1
            chosen_neighbor = first(viable_neighbors)
        else
            viable_neighbor_indices = first.(viable_neighbors)
            step_likelihoods = StatsBase.weights(edge_likelihoods[current_kmer_index, viable_neighbor_indices])
            chosen_neighbor = StatsBase.sample(viable_neighbors, step_likelihoods)
        end
        kmer = kmer_index_and_orientation_to_kmer(graph, chosen_neighbor...)
        push!(walk, kmer)
        current_kmer_indices = searchsorted(kmers, BioSequences.canonical(last(walk)))
        @assert length(current_kmer_indices) == 1
        current_kmer_index = first(current_kmer_indices)
    end
    return walk
end

take_a_walk (generic function with 1 method)

In [7]:
# create a random genome

In [8]:
L = 100

100

In [9]:
genome = BioSequences.randdnaseq(Random.seed!(L), L)

100nt DNA Sequence:
CTCACTGCAAAGCGCCCAGACCAGAGCCTCACCTACAAG…TGTACTATCTTCTCGAATACCAGCCGTCGGCCGGTATAA

In [10]:
k = 7

7

In [11]:
kmer_counts = sort!(Mycelia.count_canonical_kmers(BioSequences.BigDNAMer{k}, genome))

OrderedCollections.OrderedDict{BioSequences.BigDNAMer{7}, Int64} with 94 entries:
  AAAGCGC => 1
  AACATAG => 1
  AAGATAG => 1
  AAGCGCC => 1
  AAGTGGA => 1
  AATACCA => 1
  AATCTAT => 1
  ACAAGTG => 1
  ACAATCT => 1
  ACATAGA => 1
  ACCAGAG => 1
  ACCAGCC => 1
  ACCGCAG => 1
  ACCGGCC => 1
  ACCTACA => 1
  ACGGCTG => 1
  ACTATCT => 1
  ACTGCAA => 1
  ACTTGTA => 1
  AGAAGAT => 1
  AGACCAG => 1
  AGAGCCT => 1
  AGCCGTC => 1
  AGCCTCA => 1
  AGCGCCC => 1
  ⋮       => ⋮

In [12]:
K = length(keys(kmer_counts))

94

In [13]:
# create an undirected kmer graph from the sequence

In [14]:
graph = MetaGraphs.MetaGraph(K)

{94, 0} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [15]:
for (i, (kmer, count)) in enumerate(kmer_counts)
    @show i, kmer, count
    @show MetaGraphs.set_prop!(graph, i, :kmer, kmer)
    @show MetaGraphs.set_prop!(graph, i, :count, count)
end

(i, kmer, count) = (1, AAAGCGC, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (2, AACATAG, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (3, AAGATAG, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (4, AAGCGCC, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (5, AAGTGGA, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (6, AATACCA, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (7, AATCTAT, 1)
MetaGraphs.set_prop!(graph, i, :kmer, kmer) = true
MetaGraphs.set_prop!(graph, i, :count, count) = true
(i, kmer, count) = (8, ACAA

In [16]:
graph.vprops

Dict{Int64, Dict{Symbol, Any}} with 94 entries:
  5  => Dict(:count=>1, :kmer=>AAGTGGA)
  56 => Dict(:count=>1, :kmer=>CCTCACC)
  35 => Dict(:count=>1, :kmer=>ATGTTGA)
  55 => Dict(:count=>1, :kmer=>CCTACAA)
  60 => Dict(:count=>1, :kmer=>CGCCCAG)
  30 => Dict(:count=>1, :kmer=>AGTGGAC)
  32 => Dict(:count=>1, :kmer=>ATACCGG)
  6  => Dict(:count=>1, :kmer=>AATACCA)
  67 => Dict(:count=>1, :kmer=>CTGCAAA)
  45 => Dict(:count=>1, :kmer=>CAGAGCC)
  73 => Dict(:count=>1, :kmer=>GAGAAGA)
  64 => Dict(:count=>1, :kmer=>CGTCGGC)
  90 => Dict(:count=>1, :kmer=>TAGGTGA)
  4  => Dict(:count=>1, :kmer=>AAGCGCC)
  13 => Dict(:count=>1, :kmer=>ACCGCAG)
  54 => Dict(:count=>1, :kmer=>CCGGCCG)
  63 => Dict(:count=>1, :kmer=>CGGTCCA)
  86 => Dict(:count=>1, :kmer=>GCTGGTA)
  91 => Dict(:count=>1, :kmer=>TAGTACA)
  62 => Dict(:count=>1, :kmer=>CGGTATA)
  58 => Dict(:count=>1, :kmer=>CGACGGC)
  52 => Dict(:count=>1, :kmer=>CCGACGG)
  12 => Dict(:count=>1, :kmer=>ACCAGCC)
  28 => Dict(:count=>1, :kmer=>A

In [17]:
kmers = collect(keys(kmer_counts))

94-element Vector{BioSequences.BigDNAMer{7}}:
 AAAGCGC
 AACATAG
 AAGATAG
 AAGCGCC
 AAGTGGA
 AATACCA
 AATCTAT
 ACAAGTG
 ACAATCT
 ACATAGA
 ACCAGAG
 ACCAGCC
 ACCGCAG
 ⋮
 GCGCCCA
 GCGGTCC
 GCTCAAC
 GCTGGTA
 GGCCGAC
 GGTATAA
 GGTCCAC
 TAGGTGA
 TAGTACA
 TATTCGA
 TCGAGAA
 TCTCGAA

In [18]:
for i in 1:length(genome)-k
    edge_range = i:i+k
    edge = genome[edge_range]
    src = BioSequences.BigDNAMer{k}(edge[1:end-1])
    dst = BioSequences.BigDNAMer{k}(edge[2:end])
    canonical_src = BioSequences.canonical(src)
    canonical_dst = BioSequences.canonical(dst)
    src_index = Mycelia.get_kmer_index(kmers, canonical_src)
    dst_index = Mycelia.get_kmer_index(kmers, canonical_dst)
    @show edge
    @show src_index, src == canonical_src, dst_index, dst == canonical_dst
    graph_edge = Graphs.Edge(src_index, dst_index)
    if Graphs.has_edge(graph, graph_edge)
        current_count = graph.eprops[graph_edge][:count]
        MetaGraphs.set_prop!(graph, graph_edge, :count, current_count+1)
    else
        Graphs.add_edge!(graph, graph_edge)
        MetaGraphs.set_prop!(graph, graph_edge, :count, 1)
    end
end

edge = CTCACTGC
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (47, false, 79, false)
edge = TCACTGCA
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (79, false, 43, true)
edge = CACTGCAA
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (43, true, 18, true)
edge = ACTGCAAA
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (18, true, 67, true)
edge = CTGCAAAG
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (67, true, 68, false)
edge = TGCAAAGC
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (68, false, 78, true)
edge = GCAAAGCG
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (78, true, 38, true)
edge = CAAAGCGC
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (38, true, 1, true)
edge = AAAGCGCC
(src_index, src == canonical_src, dst_index, dst == canonical_dst) = (1, true, 4, true)
edge = AAGCGCCC
(src_index, src == canonical

In [19]:
graph

{94, 93} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [20]:
graph.eprops

Dict{Graphs.SimpleGraphs.SimpleEdge{Int64}, Dict{Symbol, Any}} with 93 entries:
  Edge 8 => 40  => Dict(:count=>1)
  Edge 42 => 90 => Dict(:count=>1)
  Edge 6 => 70  => Dict(:count=>1)
  Edge 4 => 25  => Dict(:count=>1)
  Edge 27 => 53 => Dict(:count=>1)
  Edge 63 => 84 => Dict(:count=>1)
  Edge 67 => 68 => Dict(:count=>1)
  Edge 68 => 78 => Dict(:count=>1)
  Edge 2 => 10  => Dict(:count=>1)
  Edge 7 => 34  => Dict(:count=>1)
  Edge 62 => 88 => Dict(:count=>1)
  Edge 11 => 71 => Dict(:count=>1)
  Edge 8 => 19  => Dict(:count=>1)
  Edge 59 => 93 => Dict(:count=>1)
  Edge 10 => 34 => Dict(:count=>1)
  Edge 59 => 73 => Dict(:count=>1)
  Edge 55 => 65 => Dict(:count=>1)
  Edge 16 => 50 => Dict(:count=>1)
  Edge 61 => 87 => Dict(:count=>1)
  Edge 5 => 40  => Dict(:count=>1)
  Edge 33 => 91 => Dict(:count=>1)
  Edge 1 => 4   => Dict(:count=>1)
  Edge 29 => 37 => Dict(:count=>1)
  Edge 18 => 67 => Dict(:count=>1)
  Edge 20 => 73 => Dict(:count=>1)
  ⋮             => ⋮

In [55]:
nodesize = [graph.vprops[v][:count] for v in 1:Graphs.nv(graph)]
nodelabel = [graph.vprops[v][:kmer] for v in 1:Graphs.nv(graph)]
# GraphPlot.gplot(graph, nodesize=nodesize, nodelabel=nodelabel)
GraphPlot.gplot(graph, nodesize=nodesize)

In [56]:
edge_likelihoods = determine_edge_probabilities(graph)

94×94 SparseArrays.SparseMatrixCSC{Float64, Int64} with 186 stored entries:
⡀⠈⠀⠀⠂⠀⠀⠄⠀⠀⡀⠀⢀⠀⠀⠈⢂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠈⠀⠂⠀⠠⡂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀
⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠈⠂⢄⠀⠀⠀⠀⠀⠀⠈⠄⠀⠀⠀⡀⠀⠂⠀⠀⠀
⠀⠄⠀⡀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠈⠄⠀⠀⠂⠀⠁⠀⠀⠀⡀⠄⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠑⠈⠄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠄⢀⠀⠀⠀⠀⠀⠀⠀⠡⠀⠀⠈⠑⠀⠀⡀⠀
⠀⠐⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⡀⠀⠄⠀⠂⠐⡁⠀
⡀⠀⠈⠀⠁⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠠⠠
⠈⠐⠠⠢⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀
⠀⠀⠀⠀⠀⠀⠀⠁⠐⠂⠠⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⡁⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠢⠀⠠⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠀⠄⠁⠀⠀⡀⠀⠀⡀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠑⠄⠀⠀⠀⠀⢁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠄⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢀⠀⠀⠀⠀⠀⠀⠈⠐⠀⠀⠠⠀⠄⢀⠀⠈⠂
⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⠁⢀⠀⠀⠀⠀⠀⠀⠠⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢐⠠⠑⠀⠀
⠀⠄⡀⠀⡀⠀⠀⠁⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠊⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠁⠀⠀⠂⠄⠄⡀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀
⠀⠀⠀⠐⠀⠀⠀⠁⠀⠀⠀⠀⠀⠈⠀⠐⠀⠀⠄⠠⠀⡀⠀⠀⠀⡀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⢆⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠄⢀⢀⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⢈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⢄⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠈⠀⡂⠀⠈⠀⠀⠀⠀⠀⠀⠢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡠

In [57]:
# take yen_k shortest paths

In [58]:
genome_kmers = collect(BioSequences.each(BioSequences.BigDNAMer{k}, genome))

94-element Vector{BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}}:
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(1, CTCACTG, CAGTGAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(2, TCACTGC, GCAGTGA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(3, CACTGCA, TGCAGTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(4, ACTGCAA, TTGCAGT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(5, CTGCAAA, TTTGCAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(6, TGCAAAG, CTTTGCA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(7, GCAAAGC, GCTTTGC)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(8, CAAAGCG, CGCTTTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(9, AAAGCGC, GCGCTTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(10, AAGCGCC, GGCGCTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(11, AGCGCCC, GGGCGCT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(12, GCGCCCA, TGGGCGC)
 BioSequenc

In [59]:
# make a SNP edit to the genome

In [268]:
# seed = 0
# seed = 1
# seed = 2
# seed = 3
# seed = 4
# seed = 5
# seed = 6
# seed = 7
# seed = 8
seed = 9

9

In [269]:
@show seed
Random.seed!(seed)
observation = Mycelia.observe(genome, error_rate = 0.1)

seed = 9


104nt DNA Sequence:
TTATACCGGCCGAGGCGTGGTACTTCGAGAAGATATGTA…GTTAGGTGAGGCTCTGGGCTGGGGCTGCTTTGCAGTGAG

In [270]:
alignment = BioAlignments.pairalign(BioAlignments.LevenshteinDistance(), observation, genome)

BioAlignments.PairwiseAlignmentResult{Int64, BioSequences.LongDNASeq, BioSequences.LongDNASeq}:
  distance: 59
  seq:   1 TTATACCGGCCGAG-GCGTGGTACTTCGAGAAGATATGTACAATCTATGTTGGCTGC-GG  58
            |  | | ||  || ||   | ||  | || ||   |  ||  |  | || | | || | 
  ref:   1 CT-CA-CTGCAAAGCGCCCAG-AC--C-AG-AG-CCT-CAC-CTACAAGTGGACCGCAGC  50

  seq:  59 TCCAC-TTG-TTAGGTGAGGCTCTGGGCT-GGGGCTGCTTTG-CAGT-GAG 104
           || || | | ||     |   ||| |  |    || | |  | | ||  | 
  ref:  51 TCAACATAGATTGTACTATCTTCTCGAATACCAGCCG-TCGGCCGGTATAA 100


In [271]:
reverse_alignment = BioAlignments.pairalign(BioAlignments.LevenshteinDistance(), BioSequences.reverse_complement(observation), genome)

BioAlignments.PairwiseAlignmentResult{Int64, BioSequences.LongDNASeq, BioSequences.LongDNASeq}:
  distance: 9
  seq:   1 CTCACTGCAAAGCAGCCCCAGCCCAGAGCCTCACCTAACAAGTGGACCGCAGC-CAACAT  59
           ||||||||||||| | ||||| |||||||||||||| |||||||||||||||| ||||||
  ref:   1 CTCACTGCAAAGC-G-CCCAGACCAGAGCCTCACCT-ACAAGTGGACCGCAGCTCAACAT  57

  seq:  60 AGATTGTACATATCTTCTCGAAGTACCACGCC-TCGGCCGGTATAA 104
           ||||||||| |||||||||||| ||||| ||| |||||||||||||
  ref:  58 AGATTGTAC-TATCTTCTCGAA-TACCA-GCCGTCGGCCGGTATAA 100


In [272]:
if reverse_alignment.value < alignment.value
    observation = BioSequences.reverse_complement(observation)
    alignment = reverse_alignment
    @show "flipping"
end

"flipping" = "flipping"


"flipping"

In [273]:
# convert genome into stranded path

In [274]:
# function sequence_to_
observation_as_oriented_kmers = []
observation_kmers = collect(BioSequences.each(BioSequences.BigDNAMer{k}, observation))

98-element Vector{BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}}:
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(1, CTCACTG, CAGTGAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(2, TCACTGC, GCAGTGA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(3, CACTGCA, TGCAGTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(4, ACTGCAA, TTGCAGT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(5, CTGCAAA, TTTGCAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(6, TGCAAAG, CTTTGCA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(7, GCAAAGC, GCTTTGC)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(8, CAAAGCA, TGCTTTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(9, AAAGCAG, CTGCTTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(10, AAGCAGC, GCTGCTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(11, AGCAGCC, GGCTGCT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(12, GCAGCCC, GGGCTGC)
 BioSequenc

In [275]:
genome_kmers

94-element Vector{BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}}:
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(1, CTCACTG, CAGTGAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(2, TCACTGC, GCAGTGA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(3, CACTGCA, TGCAGTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(4, ACTGCAA, TTGCAGT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(5, CTGCAAA, TTTGCAG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(6, TGCAAAG, CTTTGCA)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(7, GCAAAGC, GCTTTGC)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(8, CAAAGCG, CGCTTTG)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(9, AAAGCGC, GCGCTTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(10, AAGCGCC, GGCGCTT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(11, AGCGCCC, GGGCGCT)
 BioSequences.MerIterResult{BioSequences.BigDNAMer{7}}(12, GCGCCCA, TGGGCGC)
 BioSequenc

In [276]:
is_canonical = falses(length(observation_kmers))
kmer_index = zeros(Int, length(observation_kmers))
for (i, kmer) in enumerate(observation_kmers)
#     is_canonical[i] = kmer.fw <= kmer.bw
    canonical_kmer = BioSequences.canonical(kmer.fw)
    is_canonical[i] = kmer.fw == canonical_kmer
    kmer_index_range = searchsorted(kmers, canonical_kmer)
    if length(kmer_index_range) > 1
        @error "bad"
    elseif isempty(kmer_index_range)
        # do nothing, index is 0 to indicate not found
    else
        kmer_index[i] = first(kmer_index_range)
    end
end

In [277]:
is_canonical

98-element BitVector:
 0
 0
 1
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 0
 0
 1
 1
 0
 0
 0
 1
 0
 1
 1

In [278]:
kmer_index

98-element Vector{Int64}:
 47
 79
 43
 18
 67
 68
 78
  0
  0
  0
  0
  0
  0
  ⋮
  0
  0
  0
  0
  0
 61
 54
 14
 81
 32
 62
 88

In [279]:
proposed_path = BioSequences.BigDNAMer{k}[]
opening_missing_kmer_path_index = findfirst(x -> x == 0, kmer_index)

# read is solid!
if opening_missing_kmer_path_index == nothing
#     proposed_seq = observation
    proposed_path = [kmer.fw for kmer in observation_kmers]
    @show "perfect read"
elseif all(index -> index == 0, kmer_index)
    @show "read is trash"
    # generate a new read from the assembly to replace this one

    observation_path_length = length(observation_kmers)
    proposed_path = nothing
    proposed_path_size_discrepancy = Inf
    starting_kmer_indices = StatsBase.sample(1:Graphs.nv(graph), min(10, Graphs.nv(graph)), replace=false)
    for starting_kmer_index in starting_kmer_indices
        starting_kmer_index = rand(1:Graphs.nv(graph))
        @show starting_kmer_index
        starting_kmer = graph.vprops[starting_kmer_index][:kmer]
        if rand(Bool)
            starting_kmer = BioSequences.reverse_complement(starting_kmer)
        end
        candidate_path = take_a_walk(graph, edge_likelihoods, kmers, starting_kmer, observation_path_length)
        discrepancy = abs(length(candidate_path) - observation_path_length)
        @show discrepancy
        if discrepancy < proposed_path_size_discrepancy
            proposed_path = candidate_path
            proposed_path_size_discrepancy = discrepancy
            if proposed_path_size_discrepancy == 0
                break
            end
        end
    end
else
    if opening_missing_kmer_path_index == 1
        opening_solid_kmer = nothing
    #     closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), opening_missing_kmer_path_index+1)
    elseif opening_missing_kmer_path_index != nothing
        opening_solid_kmer_path_index = opening_missing_kmer_path_index - 1
        opening_solid_kmer = observation_kmers[opening_solid_kmer_path_index].fw
    end

    if opening_missing_kmer_path_index == nothing
        closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), 1)
    else
        closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), opening_missing_kmer_path_index+1)
    end
    if closing_solid_kmer_path_index == nothing
        @show "end is open"
        closing_solid_kmer = nothing
    else
        closing_solid_kmer = observation_kmers[closing_solid_kmer_path_index].fw
    end


    extensions = 0
    while (opening_missing_kmer_path_index != nothing) && (extensions < 5)
        extensions += 1
        @show extensions

        if (opening_solid_kmer == nothing) && (closing_solid_kmer != nothing)
            @show "missing opening"
            @show closing_solid_kmer
            inverted_closing_solid_kmer = BioSequences.reverse_complement(closing_solid_kmer)
            @show inverted_closing_solid_kmer
            walk_length = closing_solid_kmer_path_index
            chosen_walk = take_a_walk(graph, edge_likelihoods, kmers, inverted_closing_solid_kmer, walk_length)
            chosen_walk = reverse(BioSequences.reverse_complement.(chosen_walk))
            @show chosen_walk
            @show closing_solid_kmer_path_index
            @assert observation_kmers[closing_solid_kmer_path_index].fw == last(chosen_walk)
            proposed_path = chosen_walk
            opening_missing_kmer_path_index = findnext(map(x -> x == 0, kmer_index), closing_solid_kmer_path_index+1)

            if opening_missing_kmer_path_index == 1
                opening_solid_kmer = nothing
            elseif opening_missing_kmer_path_index != nothing
                need_to_add = getproperty.(observation_kmers[closing_solid_kmer_path_index+1:opening_missing_kmer_path_index-1], :fw)
                append!(proposed_path, need_to_add)
                opening_solid_kmer_path_index = opening_missing_kmer_path_index - 1
                opening_solid_kmer = observation_kmers[opening_solid_kmer_path_index].fw
                closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), opening_missing_kmer_path_index+1)
                if closing_solid_kmer_path_index == nothing
                    closing_solid_kmer = nothing
                else
                    closing_solid_kmer = observation_kmers[closing_solid_kmer_path_index].fw
                end
            end
        end

        if (opening_solid_kmer != nothing) && (closing_solid_kmer == nothing)
            @show "missing closing"
            walk_length = length(observation_kmers) - opening_solid_kmer_path_index+1
            chosen_walk = take_a_walk(graph, edge_likelihoods, kmers, opening_solid_kmer, walk_length)
            @show chosen_walk
            @assert observation_kmers[opening_solid_kmer_path_index].fw == first(chosen_walk)
            if isempty(proposed_path)
                proposed_path = getproperty.(observation_kmers[1:opening_solid_kmer_path_index], :fw)
            end
            append!(proposed_path, chosen_walk[2:end])
            opening_missing_kmer_path_index = nothing
        end

        while (opening_solid_kmer != nothing) && (closing_solid_kmer != nothing)
            @show "bubble!"
            opening_solid_kmer_index = kmer_index[opening_solid_kmer_path_index]
            closing_solid_kmer_index = kmer_index[closing_solid_kmer_path_index]

            @show opening_solid_kmer, opening_solid_kmer_path_index
            @show closing_solid_kmer, closing_solid_kmer_path_index
            @show length(proposed_path)
            
            #####################################################################################
            # DETERMINE PATH LENGTH
            #####################################################################################
            # by default, we're assuming that we want to walk the same distance as our bubble + 1
            bubble_path_length = closing_solid_kmer_path_index - opening_solid_kmer_path_index + 1

            if opening_solid_kmer_index != closing_solid_kmer_index
                path = Graphs.a_star(graph, opening_solid_kmer_index, closing_solid_kmer_index)
                normalized_path = Int[path[1].src, [edge.dst for edge in path]...]
                shortest_paths = [normalized_path]
            else
                neighbors = Graphs.neighbors(graph, opening_solid_kmer_index)
                shortest_paths = Vector{Vector{Int}}()
                for neighbor in neighbors
                    path = Graphs.a_star(graph, neighbor, closing_solid_kmer_index)
                    normalized_path = Int[path[1].src, [edge.dst for edge in path]...]
                    push!(shortest_paths, normalized_path)
                end
            end
            # but if our bubble is a deletion, then walking that length may not get us across to the other side
            # so we also determine the longest, shortest path
            longest_shortest_walk_length = maximum(length.(shortest_paths))


            longest_walk_length = max(bubble_path_length, longest_shortest_walk_length)
            walk_length = Int(ceil(longest_walk_length * 1.1))

            walks = Vector{Int}[]
            for i in 1:3
                println("round $i")
                walks = [take_a_walk(graph, edge_likelihoods, kmers, opening_solid_kmer, walk_length)]
                while allunique(walks)
                    push!(walks, take_a_walk(graph, edge_likelihoods, kmers, opening_solid_kmer, walk_length))
                end

                walks = filter(path -> closing_solid_kmer in path, walks)

                if isempty(walks)
                    walk_length = Int(ceil(walk_length * 1.1))
                else
                    # done!
                    break
                end
            end

            if !isempty(walks)
                candidate_paths = sort(collect(StatsBase.countmap(walks)), by=x->x[2], rev=true)
                selected_path = first(first(candidate_paths))
                selected_path_closing_solid_kmer_intersects = findall(kmer -> kmer == closing_solid_kmer, selected_path)
                if length(selected_path_closing_solid_kmer_intersects) > 1
                    @show "here"
                    selected_path_closing_solid_kmer_intersect = 0
                    d = Inf
                    for i in selected_path_closing_solid_kmer_intersects
                        candidate_distance = abs(i - walk_length)
                        if candidate_distance < d
                            d = candidate_distance
                            selected_path_closing_solid_kmer_intersect = i
                        end
                    end
                else
                    selected_path_closing_solid_kmer_intersect = first(selected_path_closing_solid_kmer_intersects)
                end
                selected_path = selected_path[1:selected_path_closing_solid_kmer_intersect]
                if isempty(proposed_path)
                    proposed_path = getproperty.(observation_kmers[1:opening_solid_kmer_path_index], :fw)
                end
                append!(proposed_path, selected_path[2:end])
                opening_missing_kmer_path_index = findnext(map(x -> x == 0, kmer_index), closing_solid_kmer_path_index+1)
                if opening_missing_kmer_path_index == nothing
                    #done
                    opening_solid_kmer = nothing
                elseif opening_missing_kmer_path_index == 1
                    # also done?
                    opening_solid_kmer = nothing
                elseif opening_missing_kmer_path_index != nothing
                    need_to_add = getproperty.(observation_kmers[closing_solid_kmer_path_index+1:opening_missing_kmer_path_index-1], :fw)
                    append!(proposed_path, need_to_add)
                    opening_solid_kmer_path_index = opening_missing_kmer_path_index - 1
                    opening_solid_kmer = observation_kmers[opening_solid_kmer_path_index].fw
                    closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), opening_missing_kmer_path_index+1)
                    if closing_solid_kmer_path_index == nothing
                        closing_solid_kmer = nothing
                    else
                        closing_solid_kmer = observation_kmers[closing_solid_kmer_path_index].fw
                    end
                end
            else
                @show "need to bump out"
                if opening_missing_kmer_path_index >= 2
                    opening_missing_kmer_path_index -= 1
                    opening_solid_kmer = nothing
                    # this should now break out to larger while loop
                end
                # push out closing node and try again
                closing_solid_kmer_path_index = findnext(map(x -> x != 0, kmer_index), closing_solid_kmer_path_index+1)

                if closing_solid_kmer_path_index == nothing
                    @show "end is open"
                    closing_solid_kmer = nothing
                else
                    closing_solid_kmer = observation_kmers[closing_solid_kmer_path_index].fw
                end
            end
        end
    end
    if closing_solid_kmer_path_index != nothing
        append!(proposed_path, getproperty.(observation_kmers[closing_solid_kmer_path_index+1:end], :fw))
    end
end

extensions = 1
"bubble!" = "bubble!"
(opening_solid_kmer, opening_solid_kmer_path_index) = (GCAAAGC, 7)
(closing_solid_kmer, closing_solid_kmer_path_index) = (GCCCAGA, 21)
length(proposed_path) = 0
round 1
"bubble!" = "bubble!"
(opening_solid_kmer, opening_solid_kmer_path_index) = (GCCCAGA, 21)
(closing_solid_kmer, closing_solid_kmer_path_index) = (CCAGAGC, 23)
length(proposed_path) = 14
round 1
"bubble!" = "bubble!"
(opening_solid_kmer, opening_solid_kmer_path_index) = (TCACCTA, 31)
(closing_solid_kmer, closing_solid_kmer_path_index) = (ACAAGTG, 38)
length(proposed_path) = 29
round 1
"bubble!" = "bubble!"
(opening_solid_kmer, opening_solid_kmer_path_index) = (CCGCAGC, 47)
(closing_solid_kmer, closing_solid_kmer_path_index) = (CAACATA, 54)
length(proposed_path) = 44
round 1
"bubble!" = "bubble!"
(opening_solid_kmer, opening_solid_kmer_path_index) = (ATTGTAC, 62)
(closing_solid_kmer, closing_solid_kmer_path_index) = (TATCTTC, 70)
length(proposed_path) = 60
round 1
"bubble!" = "bubble!"


94-element Vector{BioSequences.BigDNAMer{7}}:
 CTCACTG
 TCACTGC
 CACTGCA
 ACTGCAA
 CTGCAAA
 TGCAAAG
 GCAAAGC
 CAAAGCG
 AAAGCGC
 AAGCGCC
 AGCGCCC
 GCGCCCA
 CGCCCAG
 ⋮
 AGCCGTC
 GCCGTCG
 CCGTCGG
 CGTCGGC
 GTCGGCC
 TCGGCCG
 CGGCCGG
 GGCCGGT
 GCCGGTA
 CCGGTAT
 CGGTATA
 GGTATAA

In [280]:
proposed_path

94-element Vector{BioSequences.BigDNAMer{7}}:
 CTCACTG
 TCACTGC
 CACTGCA
 ACTGCAA
 CTGCAAA
 TGCAAAG
 GCAAAGC
 CAAAGCG
 AAAGCGC
 AAGCGCC
 AGCGCCC
 GCGCCCA
 CGCCCAG
 ⋮
 AGCCGTC
 GCCGTCG
 CCGTCGG
 CGTCGGC
 GTCGGCC
 TCGGCCG
 CGGCCGG
 GGCCGGT
 GCCGGTA
 CCGGTAT
 CGGTATA
 GGTATAA

In [281]:
proposed_seq = BioSequences.LongDNASeq(first(proposed_path))
for kmer in proposed_path[2:end]
    push!(proposed_seq, last(kmer))
end
proposed_seq

100nt DNA Sequence:
CTCACTGCAAAGCGCCCAGACCAGAGCCTCACCTACAAG…TGTACTATCTTCTCGAATACCAGCCGTCGGCCGGTATAA

In [282]:
new_alignment = BioAlignments.pairalign(BioAlignments.LevenshteinDistance(), proposed_seq, genome)

BioAlignments.PairwiseAlignmentResult{Int64, BioSequences.LongDNASeq, BioSequences.LongDNASeq}:
  distance: 0
  seq:   1 CTCACTGCAAAGCGCCCAGACCAGAGCCTCACCTACAAGTGGACCGCAGCTCAACATAGA  60
           ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
  ref:   1 CTCACTGCAAAGCGCCCAGACCAGAGCCTCACCTACAAGTGGACCGCAGCTCAACATAGA  60

  seq:  61 TTGTACTATCTTCTCGAATACCAGCCGTCGGCCGGTATAA 100
           ||||||||||||||||||||||||||||||||||||||||
  ref:  61 TTGTACTATCTTCTCGAATACCAGCCGTCGGCCGGTATAA 100


In [283]:
reverse_new_alignment = BioAlignments.pairalign(BioAlignments.LevenshteinDistance(), BioSequences.reverse_complement(proposed_seq), genome)

BioAlignments.PairwiseAlignmentResult{Int64, BioSequences.LongDNASeq, BioSequences.LongDNASeq}:
  distance: 58
  seq:   1 TTATACCGGCCGA-CGGCTGGTATTCGAGAAGATAGTACA-ATCTATGTTGAGCTGC-GG  57
            |  | | ||  | || |  | |  | || ||    | ||  |  | || || | || | 
  ref:   1 CT-CA-CTGCAAAGCGCCCAG-A--CCAG-AG--CCT-CACCTACAAGTGGA-CCGCAGC  50

  seq:  58 TCCAC-TTG-TAG-GTGAGGCTCT-G-GT-CTGGGCGCTTTG-CAGT-GAG 100
           || || | | | |    |   ||| |  | |  | || |  | | ||  | 
  ref:  51 TCAACATAGATTGTACTATCTTCTCGAATACCAGCCG-TCGGCCGGTATAA 100


In [284]:
if reverse_new_alignment.value < new_alignment.value
#     observation = BioSequences.reverse_complement!(observation)
    new_alignment = reverse_new_alignment
    @show "flipping"
end

In [285]:
if new_alignment.value <= alignment.value
    @show "newer is equivalent or better"
    @show new_alignment.value, alignment.value
else
    @show "newer is worse, keep original"
    @show new_alignment.value, alignment.value
end

"newer is equivalent or better" = "newer is equivalent or better"
(new_alignment.value, alignment.value) = (0, 9)


(0, 9)